In [1]:
import os
import sys
# Add src/ to sys.path (relative to current notebook)
sys.path.append(os.path.abspath("../../src"))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pathlib
import mlflow

from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.decomposition import PCA
from sklearn.linear_model import Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neural_network import  MLPRegressor
from xgboost import XGBRegressor
from tqdm import tqdm 


from utils.VariableAnalysis import UnivariateAnalysis,BivariateAnalysis

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns',None)

In [2]:
file_path = pathlib.Path('D:\Capstone Project\dataset\House_Price_dataset')
df = pd.read_csv(f"{file_path}/10.gurgaon_properties_post_feature_selection_v2.csv")
df.head()

,property_type,sector,price,bedRoom,bathroom,balcony,agePossession,built_up_area,servant room,store room,furnishing_type,luxury_category,floor_category
0,flat,sector 36,0.82,3,2,2,New Property,850.0,0,0,0,Low,Low Floor
1,flat,sector 89,0.95,2,2,2,New Property,1226.0,1,0,0,Low,Mid Floor
2,flat,sohna road,0.32,2,2,1,New Property,1000.0,0,0,0,Low,High Floor
3,flat,sector 92,1.60,3,4,3+,Relatively New,1615.0,1,0,1,High,Mid Floor
4,flat,sector 102,0.48,2,2,1,Relatively New,582.0,0,1,0,High,Mid Floor


In [3]:
## Furniture Type Unique Value
df['furnishing_type'].value_counts()

furnishing_type
0    2349
1    1018
2     187
Name: count, dtype: int64

In [4]:
## Level Incoding 
# 0 -> unfurnished
# 1 -> semifurnished
# 2 -> furnished
df['furnishing_type'] = df['furnishing_type'].replace({0.0:'unfurnished',1.0:'semifurnished',2.0:'furnished'})
df['furnishing_type'].value_counts()

furnishing_type
unfurnished      2349
semifurnished    1018
furnished         187
Name: count, dtype: int64

In [5]:
## Data split into dependent variable and Independent variable 
X = df.drop(columns=['price'])
y = df['price']

In [6]:
# Applying the log1p transformation to the target variable for normal distribution
y_log_tran = np.log1p(y)

## 1. Ordinal Encoding Approach for Categoriacal Value
- Pipe Line Preparation and Test Run

In [7]:
columns_to_encode = ['property_type','sector', 'balcony', 'agePossession', 'furnishing_type', 'luxury_category', 'floor_category']

In [8]:
## Single Model Test Run
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('numerical_transform', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('categorical_tranform', OrdinalEncoder(), columns_to_encode)
    ], 
    remainder='passthrough'
)

# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

scores = cross_val_score(pipeline, X, y_log_tran, cv=kfold, scoring='r2')
print(f"Score Mean:{scores.mean()}, Score std:{scores.std()}")

## Train and Test split and Model train 
X_train, X_test, y_train, y_test = train_test_split(X,y_log_tran,test_size=0.2,random_state=42)
print(pipeline.fit(X_train,y_train))

y_pred = pipeline.predict(X_test)
y_pred = np.expm1(y_pred) ## Reversing Log normal tranformation into Original Scale applied in the Target Feature
mabe = mean_absolute_error(np.expm1(y_test),y_pred)
print(f"mean_absolute_error:  {mabe}")

Score Mean:0.7363096633436828, Score std:0.03238005754429938
Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('numerical_transform',
                                                  StandardScaler(),
                                                  ['bedRoom', 'bathroom',
                                                   'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('categorical_tranform',
                                                  OrdinalEncoder(),
                                                  ['property_type', 'sector',
                                                   'balcony', 'agePossession',
                                                   'furnishing_type',
                                                   'luxury_c

In [9]:
# ### Original Script 
# def scorer(model_name, model, y_transformed=y_log_tran):
#     """Function For Different Models """
#     output = []
#     output.append(model_name)

#     pipeline = Pipeline([
#         ('preprocessor', preprocessor),
#         ('regressor', model)
#     ])

#     # K-fold cross-validation (Model Evaluation)
#     kfold = KFold(n_splits=10, shuffle=True, random_state=42)
#     scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
#     output.append(round(scores.mean(),4))
    
#     X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)   ## Train Test Split
#     pipeline.fit(X_train,y_train) ## Model Training 
#     y_pred = pipeline.predict(X_test) ## y^ Calulation 
#     y_pred = np.expm1(y_pred) ## Reversing Log normal tranformation into Original Scale in the Target Feature
#     output.append(mean_absolute_error(np.expm1(y_test),y_pred))
#     return output

# ### Models Dictionary
# model_dict = {
#     'LinearRegression':LinearRegression(),
#     'SVR':SVR(),
#     'Ridge':Ridge(),
#     'Lasso':Lasso(),
#     'DecisionTreeRegressor': DecisionTreeRegressor(),
#     'RandomForestRegressor':RandomForestRegressor(),
#     'ExtraTreesRegressor': ExtraTreesRegressor(),
#     'GradientBoostingRegressor': GradientBoostingRegressor(),
#     'AdaBoostRegressor': AdaBoostRegressor(),
#     'MLPRegressor': MLPRegressor(),
#     'XGBRegressor':XGBRegressor()
# }


# # Creating a column transformer for preprocessing
# preprocessor = ColumnTransformer(
#     transformers=[
#         ('numerical_transform', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
#         ('categorical_tranform', OrdinalEncoder(), columns_to_encode)
#     ], 
#     remainder='passthrough'
# )

# ## Function Call for all Linear Models in dictionary 
# model_output = []
# for model_name,model in tqdm(model_dict.items()):
#     model_output.append(scorer(model_name, model))

# model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])
# model_df.sort_values(['mae'])


In [10]:
## Modified Script For MLFLOW Tracking 
def scorer(model_name, model, y_transformed=y_log_tran):
    """Function For Different Models """
    Mlflow_info = {}
    transformer_info = {}
    cross_val_info = {}
    metric_info = {}
    output = []
    output.append(model_name)
    
    pipeline = Pipeline([
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    for name, transformer, columns in preprocessor.transformers:
        transformer_info[name]=transformer
       
    # K-fold cross-validation (Model Evaluation)
    kfold = KFold(n_splits=10, shuffle=True, random_state=42)
    scores = cross_val_score(pipeline, X, y_transformed, cv=kfold, scoring='r2')
    metric_info['R2'] = scores.mean() ## mlflow info
    output.append(scores.mean())
    
    X_train, X_test, y_train, y_test = train_test_split(X,y_transformed,test_size=0.2,random_state=42)   ## Train Test Split
    pipeline.fit(X_train,y_train) ## Model Training 
    y_pred = pipeline.predict(X_test) ## y^ Calulation 
    y_pred = np.expm1(y_pred) ## Reversing Log normal tranformation into Original Scale in the Target Feature
    metric_info['MAE'] = mean_absolute_error(np.expm1(y_test),y_pred) ## mlflow info
    output.append(mean_absolute_error(np.expm1(y_test),y_pred))

    ## Mlflow Parameter 
    # Mlflow_info[model_name] = model
    Mlflow_info['transformers'] = transformer_info
    Mlflow_info['kfold_params'] = {
    'n_splits': kfold.n_splits,
    'shuffle': kfold.shuffle,
    'random_state': kfold.random_state}
    Mlflow_info['metric'] = metric_info
    return output, Mlflow_info




In [11]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('numerical_transform', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('categorical_tranform', OrdinalEncoder(), columns_to_encode)
    ], 
    remainder='passthrough'
)

### Models Dictionary
model_dict = {
    'LinearRegression':LinearRegression(),
    'SVR':SVR(),
    'Ridge':Ridge(),
    'Lasso':Lasso(),
    'DecisionTreeRegressor': DecisionTreeRegressor(),
    'RandomForestRegressor':RandomForestRegressor(),
    'ExtraTreesRegressor': ExtraTreesRegressor(),
    'GradientBoostingRegressor': GradientBoostingRegressor(),
    'AdaBoostRegressor': AdaBoostRegressor(),
    'MLPRegressor': MLPRegressor(),
    'XGBRegressor':XGBRegressor()
}


## Function Call for all Linear Models in dictionary 
model_output = []
mlflow_experiment = {}
for model_name,model in tqdm(model_dict.items()):
    a,b = scorer(model_name, model)
    model_output.append(a)
    mlflow_experiment[model_name] = b
model_output

100%|██████████| 11/11 [00:20<00:00,  1.87s/it]


[['LinearRegression', np.float64(0.7363096633436828), 0.9463822160089355],
 ['SVR', np.float64(0.7642012011196353), 0.8472636473483927],
 ['Ridge', np.float64(0.7363125343993554), 0.9463387741853388],
 ['Lasso', np.float64(0.05943378064493572), 1.528905986892753],
 ['DecisionTreeRegressor', np.float64(0.7725171222824466), 0.7267948206391929],
 ['RandomForestRegressor', np.float64(0.8809331910181447), 0.5366778875359599],
 ['ExtraTreesRegressor', np.float64(0.8674328386636239), 0.5536754030442045],
 ['GradientBoostingRegressor',
  np.float64(0.8726544629211398),
  0.5768048696893772],
 ['AdaBoostRegressor', np.float64(0.7522412772899942), 0.7982813145872494],
 ['MLPRegressor', np.float64(0.8097136524705639), 0.7373221935870486],
 ['XGBRegressor', np.float64(0.8894876835260124), 0.5040475141482346]]

In [12]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])
model_df.sort_values(['mae'])

,name,r2,mae
10,XGBRegressor,0.889488,0.504048
5,RandomForestRegressor,0.880933,0.536678
6,ExtraTreesRegressor,0.867433,0.553675
7,GradientBoostingRegressor,0.872654,0.576805
4,DecisionTreeRegressor,0.772517,0.726795
9,MLPRegressor,0.809714,0.737322
8,AdaBoostRegressor,0.752241,0.798281
1,SVR,0.764201,0.847264
2,Ridge,0.736313,0.946339
0,LinearRegression,0.736310,0.946382


## 2. One Hot encoding Approach for Categoriacal Value
- Pipe Line Preparation and Test Run in Regression Models

In [14]:
### Single Modle Test Run

# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first'),['sector','agePossession','furnishing_type'])
    ], 
    remainder='passthrough'
)

# Creating a pipeline
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

# K-fold cross-validation
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(pipeline, X, y_log_tran, cv=kfold, scoring='r2')
print(f"Score Mean:{scores.mean()}, Score std:{scores.std()}")


X_train, X_test, y_train, y_test = train_test_split(X,y_log_tran,test_size=0.2,random_state=42)
print(pipeline.fit(X_train,y_train))

y_pred = pipeline.predict(X_test)
y_pred = np.expm1(y_pred) ## Reversing Log normal tranformation into Original Scale applied in the Target Feature
mabe = mean_absolute_error(np.expm1(y_test),y_pred)
print(f"mean_absolute_error:  {mabe}")

Score Mean:0.8546054073648314, Score std:0.01599847663314007
Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('num', StandardScaler(),
                                                  ['bedRoom', 'bathroom',
                                                   'built_up_area',
                                                   'servant room',
                                                   'store room']),
                                                 ('cat', OrdinalEncoder(),
                                                  ['property_type', 'sector',
                                                   'balcony', 'agePossession',
                                                   'furnishing_type',
                                                   'luxury_category',
                                                   'floor_category']),
                                                 ('c

In [15]:
# Creating a column transformer for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['bedRoom', 'bathroom', 'built_up_area', 'servant room', 'store room']),
        ('cat', OrdinalEncoder(), columns_to_encode),
        ('cat1',OneHotEncoder(drop='first'),['sector','agePossession','furnishing_type'])
    ], 
    remainder='passthrough'
)

### Models Dictionary
model_dict = {
    'LinearRegression':LinearRegression(),
    'SVR':SVR(),
    'Ridge':Ridge(),
    'Lasso':Lasso(),
    'DecisionTreeRegressor': DecisionTreeRegressor(),
    'RandomForestRegressor':RandomForestRegressor(),
    'ExtraTreesRegressor': ExtraTreesRegressor(),
    'GradientBoostingRegressor': GradientBoostingRegressor(),
    'AdaBoostRegressor': AdaBoostRegressor(),
    'MLPRegressor': MLPRegressor(),
    'XGBRegressor':XGBRegressor()
}

## Function Call for all Models in dictionary 
model_output = []
mlflow_experiment = {}
for model_name,model in tqdm(model_dict.items()):
    a,b = scorer(model_name, model)
    model_output.append(a)
    mlflow_experiment[model_name] = b
model_output

100%|██████████| 11/11 [02:02<00:00, 11.15s/it]


[['LinearRegression', np.float64(0.8546054073648314), 0.6497382874070646],
 ['SVR', np.float64(0.7697413260547326), 0.8341243500492146],
 ['Ridge', np.float64(0.8546783310939278), 0.65291434945168],
 ['Lasso', np.float64(0.05943378064493578), 1.528905986892753],
 ['DecisionTreeRegressor', np.float64(0.8074978250257523), 0.7054559381367012],
 ['RandomForestRegressor', np.float64(0.8900786519602379), 0.4928216759869119],
 ['ExtraTreesRegressor', np.float64(0.8942296764446465), 0.46158279910784134],
 ['GradientBoostingRegressor',
  np.float64(0.8765999202833739),
  0.5703766564243697],
 ['AdaBoostRegressor', np.float64(0.7573556214950709), 0.847319433802635],
 ['MLPRegressor', np.float64(0.8734573603009437), 0.5363171098624728],
 ['XGBRegressor', np.float64(0.8958499681743852), 0.4934562667923469]]

In [16]:
model_df = pd.DataFrame(model_output, columns=['name','r2','mae'])
model_df.sort_values(['mae'])

,name,r2,mae
6,ExtraTreesRegressor,0.894230,0.461583
5,RandomForestRegressor,0.890079,0.492822
10,XGBRegressor,0.895850,0.493456
9,MLPRegressor,0.873457,0.536317
7,GradientBoostingRegressor,0.876600,0.570377
0,LinearRegression,0.854605,0.649738
2,Ridge,0.854678,0.652914
4,DecisionTreeRegressor,0.807498,0.705456
1,SVR,0.769741,0.834124
8,AdaBoostRegressor,0.757356,0.847319
